In [57]:
#Moisture evolution (5 minutes)
import numpy as np
import pandas as pd
import random
from datetime import date
from datetime import datetime
from datetime import timedelta
from eto import ETo #For daily evapotranspiration calculation

    
def Kcgenerator(Kcini,Kcmed,Kcend,dini,dcob,dmed,dend):
    global daystot
    global Kc
    #Kc array creation
    daystot=dini+dcob+dmed+dend
    time = np.arange(daystot)
    inivec = np.zeros(dini) 
    inivec = inivec + Kcini
    cobvec=np.arange(Kcini, Kcmed, (Kcmed-Kcini)/dcob)
    medvec = np.zeros(dmed) 
    medvec = medvec + Kcmed
    endvec=np.arange(Kcmed, Kcend, -(Kcmed-Kcend)/dend)
    Kc=np.concatenate([inivec,cobvec,medvec,endvec])
    #Only for vineyards
    #restvec = np.zeros(365-daystot)
    #restvec = restvec + Kcini
    #Kc=np.concatenate([inivec,cobvec,medvec,endvec,restvec])
    #daystot=365

def timevecgenerator(initial_date):
    global timevec
    global time5min
    #global initial_date
    timevec=[]
    time5min = np.arange(daystot*24*12)#Point every 5min
    for i in range(0, len(time5min)):
        initial_date = initial_date + timedelta(minutes=5)
        timevec.append(initial_date)

def KsKigenerator():
    global Ks
    global Ki
    #Ks calculations
    PR=int((FC+PWP)/2)
    PGAP=int(abs(100*(PR-PWP)))
    Kss1=np.ones(int(100*PWP))*0.1
    Kss2=np.arange(0.1,1,0.9/(PGAP))
    Kss3=np.ones(100-100*PR)
    Ks=np.concatenate([Kss1,Kss2,Kss3])

#_____OLD____________
#     ns=(round((EF)*100))
#     Ks1=np.arange(0, 1, (1/EF/100))
#     Ks1=Ks1**3#Chapuza
#     Ks2=np.ones(100-ns)
#     Ks=np.concatenate([Ks1,Ks2])

    #Ki calculations
    #ni=(round((IT)*100)-6)
    FClimit=int(FC*100)
    ni2=100*(SAT-FC) #20
    Ki1=np.ones(FClimit) #
    #Ki2=np.arange(1, 50, 1/6)
    Ki4=np.arange(1,ni2,1)
    Ki4=Ki4*0.5
    Ki4=Ki4**2
    Ki4=Ki4+0.75
    #Ki2=np.array([2,10,30,40,50,64])
    #Ki2=Ki2**IF
    Ki3=np.ones(100-int(SAT*100))*7
    Ki=np.concatenate([Ki1,Ki4,Ki3]) #We add ones just in case

def raingenerator():
    global rainvec
    rainvec=np.zeros(len(time5min))#Maybe this can be improved
    raindays=random.randint(1,15)
    rainmoments=random.sample(range(1, daystot*24*12-50), raindays)#-50 is in the case the output is close to the end
    for i in range(0,raindays):
        rainlen=random.randint(1,36)#Duration
        rainvol=random.random()#Volume
        for j in range(0,rainlen):
            rainvec[rainmoments[i]+j]=rainvol/50

def samplingvecgenerator(SI):
    global samplingvec
    samplingvec=[]
    interval = round((SI/5)-1)
    i=0
    while (i < len(time5min)):
        samplingvec.append(1)
        i += 1
        for j in range(interval):
            samplingvec.append(0)
            i += 1
            
def tdmin(delayacumtime):
    days, hours, minutes = delayacumtime.days, delayacumtime.seconds // 3600, delayacumtime.seconds // 60 % 60
    minutestot=days*24*60+hours*60+minutes
    return minutestot

def crop_init(crop):
    Kcgenerator(crop["Kcini"], crop["Kcmed"], crop["Kcend"],crop["dini"],crop["dgrow"],crop["dmed"],crop["dend"])
    #Kc(Kcini,Kcmed,Kcend,dini,dcob,dmed,dend)
    
def ET0generator(weather,initial_date_in):
    final_date= initial_date_in + timedelta(days=daystot+1)
    global ET
    et1 = ETo()
    inidate = initial_date_in.strftime("%Y-%m-%d")
    endate = final_date.strftime("%Y-%m-%d")
    #inidate=datetime.strptime(inidate,"%Y-%m-%d")
    #endate=datetime.strptime(endate,"%Y-%m-%d")
    et1.param_est(weather[inidate:endate], freq, z_msl, lat, lon, TZ_lon, z_u, K_rs=0.16, a_s=0.25, b_s=0.5, alb=0.23)
    #et1.param_est(weather['2010-04-01':'2010-04-30'], freq, z_msl, lat, lon, TZ_lon, z_u, K_rs=0.16, a_s=0.25, b_s=0.5, alb=0.23)
    ET=et1.eto_fao(max_ETo=15, min_ETo=0, interp=True, maxgap=15)

def simulation():
    global readingvec
    global policyquality
    global wcvec
    global delay_wc_total
    global delay_min_total
    global irrigationnumber
    global wcQ
    global timeQ
    global Year_stress
    wc=initial_wc
    delay_wc_total=0 #Reset accumulated wc delay
    irrigationnumber=0
    delay_min_total=initial_date-initial_date#IMPROVE, Reset accumulated time delay
    readingvec = []
    wcvec=[]
    Total_stress=[]
    Year_stress=0
    irrigationflag=1
    opmoment=0
    #readingvec.append(-1)#to solve not concordance problem
    for i in range(0, len(time5min)):
        wcindex=int(round((wc)*100)-1)#Avoids extra calculations
        dateindex=int(round(i/(288))-1)#Obtains the day, avoids calculations
        wc=wc-(ET[dateindex]/11520)*Kc[dateindex]*Ki[wcindex]*Ks[wcindex]
        #wc=wc-ET0*Kc[dateindex]*Ki[wcindex]*Ks[wcindex]
        #wc=wc-ET0*Kc[int(round(i/(288))-1)]*Ks[int(round((wc)*100)-1)]*Ki[int(round((wc)*100)-1)]
        wcvec.append(wc)
        if wc>SAT:
            wc=SAT
        #late irrigation, stores the closest value to irrigation target
        if (wc <= wcthreshold) and irrigationflag:
            irrigationflag=0
            if (abs(wcthreshold-wcvec[i]))<(abs(wcthreshold-wcvec[i-1])):
                opmoment=timevec[i]
            else:
                opmoment=timevec[i-1]
        #Sampling:
        if samplingvec[i]==1:
            readingvec.append(wc)
#___________Smart policy______________            
#             if (wc>ahorrolevel1):#Saving polcies
#                 samplingvec[int(i+SI/5)]=0 #Avoid future readings
#                 if (wc>ahorrolevel2):
#                     samplingvec[int(i+2*SI/5)]=0
#                     samplingvec[int(i+3*SI/5)]=0            
#___________Irrigation________________
            if (wc <= wcthreshold):
                irrigationnumber += 1
                delay_wc_now = wcthreshold-wc #Actual wc delay
                delay_min_now = timevec[i]-opmoment
                delay_wc_total = delay_wc_total + delay_wc_now #Accumulated delay in WC
                delay_min_total = delay_min_total+(timevec[i]-opmoment) #Accumulated delay in numero feo(min)
                wc=IT #Watering until target
                irrigationflag=1
                
                #ACTUAL STRESS DUE TO IRRIGATION DELAY
                wcQ_now=1-(delay_wc_now)/((FC-PWP)/2)
                #wcQ_now=wcQ_now**6 #Compensation
                wcQ_now = wcQ_now**6
                delay_min_now=tdmin(delay_min_now)
                timeQ_now=1-(((delay_min_now)/60)/12)
                Stress_now=1-(wcQ_now*timeQ_now)
                Stress_now_compensated = Stress_now * color_map.loc[round((1-Stress_now)*100),"Factor"]
                #Stress_now_compensated = Stress_now_compensated/150
                #Stress_now_compensated = Stress_now_compensated**(0.1)
                Total_stress.append(Stress_now_compensated)
                #print(Stress_now,Stress_now_compensated)
                #print("Regamos loco(WC-T)",wcQ_now,timeQ_now,50000*delay_wc_now,delay_min_now,Stress_now)
        else:
            readingvec.append(-1)#Chapuza, mejorar para no representra elementos nulos
#_____Old method to calculate stress (Season average)_______         
    delay_min_total=tdmin(delay_min_total)#Total delay in minutes
    policyquality = 1-((delay_wc_total*(delay_min_total))/irrigationnumber**2)
    Year_stress = np.mean(Total_stress)
    wcQ=1-((delay_wc_total)/irrigationnumber)/((FC-PWP)/2)
    wcQ=wcQ**6 #Compensation
    timeQ=1-(((delay_min_total/irrigationnumber)/60)/12)
    #Stress=wcQ*timeQ
    
#____________________________________Parameters____________________________________________________________________
lettuce = {'plant_date': datetime(2010, 4, 1, 00, 00, 00),
  'dini': 20, 'dgrow': 30,'dmed':15,'dend':10, 'Kcini': 0.45 , 'Kcmed': 1 , 'Kcend': 0.9, 'name_map':'color_map_lettuce.csv',
  'file_name':'almeria10-19.csv'}
corn = {'plant_date': datetime(2010, 4, 1, 00, 00, 00),
  'dini': 30, 'dgrow': 40,'dmed':50,'dend':30, 'Kcini': 0.35 , 'Kcmed': 1.2 , 'Kcend': 0.35, 'name_map':'color_map_corn.csv',
  'file_name':'Huesca10-19.csv'}
vineyard = {'plant_date': datetime(2010, 3, 15, 00, 00, 00),
  'dini': 30, 'dgrow': 60,'dmed':40,'dend':80, 'Kcini': 0.3 , 'Kcmed': 0.7 , 'Kcend': 0.45'name_map':'color_map_vineyard.csv',
  'file_name':'Tomelloso10-19.csv'}
#_________General Parameters___________
initial_wc = 0.3 #Initial water content
wcthreshold = 0.2 #Irrigation threshold
IT = 0.4 #Irrigation target
ET0 = 0.1 #Daily nominal EvapoTranspiration in VWC% (ex: hibiscus=0.1)
ET0=ET0/(24*12)
SI = 120#sampling interval in minutes, should be multiple of 5

#____SOIL PARAMETERS-VWC(0-1)__________
SAT=0.45 #Saturation level
FC=0.35 #Field capacity
PWP=0.14 #Permanent wilting point

#_____Smart sensing parameters_________
ahorrolevel1=wcthreshold+0.012
ahorrolevel2=wcthreshold+0.04

#_______Parameters for ETo_____________
z_msl = 700 #Elevation of the met station above mean sea level (m) (only needed if P is not in df).
z_u = 2
lat = 39.14 #Latitud
lon = -2.94 #Longitud
TZ_lon = -2.94 # The longitude of the center of the time zone (dec deg) (only needed if calculating ETo hourly)
freq = 'D' #D for daily calculations

#____________________________________MAIN________________________________________________________________

crop=lettuce #Set type of crop
Kcgenerator(crop["Kcini"], crop["Kcmed"], crop["Kcend"],crop["dini"],crop["dgrow"],crop["dmed"],crop["dend"])
initial_date = crop["plant_date"] #Full format, includes hour
timevecgenerator(initial_date)
#raingenerator()
KsKigenerator()
samplingvecgenerator(SI)
weather=pd.read_csv("almeria10-19.csv")
weather['datetime'] = pd.to_datetime(weather['datetime'], format="%Y-%m-%d")
weather.set_index('datetime',inplace=True)
ET0generator(weather,initial_date)
color_map=pd.read_csv(crop['name_map'])
color_map=color_map.set_index("Stress", drop = False)


# simulation()
# delaymin=tdmin(delaytime)
# plotter2(timevec,wcvec,wcthreshold,readingvec)

# powerr = samplingvec.count(1)
# print("Sampling interval:",SI," Quality:",policyquality," Energy used:",powerr)
# print("Irrigation events",irrigationnumber, "Total min delay", delay_min_total, "Avergae min delay", delay_min_total/irrigationnumber,
#       "Total wc delay",delay_wc_total,"Average wc delay",delay_wc_total/irrigationnumber)
# print("WCQ",wcQ,"timeQ",timeQ,"Stress",Stress)


#_________________Several years function_______________________
SIvec = [5,10,20,30,40,50,60,75,90,120,150,180]
Qvec_acum = []
Energyvec = []
Ratiovec = []
for j in range(len(SIvec)):#Recorre different Sampling intervals
    Qvec=[]
    SI=SIvec[j]
    initial_date = crop["plant_date"]
    samplingvecgenerator(SI)
    for i in range (10):#Recorre differet years
        year=2010+i
        timevecgenerator(initial_date)
        ET0generator(weather,initial_date)
        simulation()
        #delaymin=tdmin(delaytime)
        Qvec.append(Year_stress)
        #print(year,"Irrigation events:",irrigationnumber,"Policy quality:", policyquality,", Delay wc:", delaywc,", Delay minutes:", delaymin)
        initial_date = initial_date + timedelta(days=365)
        print(year,SI,Year_stress,delay_min_total/irrigationnumber)
    energy=(5/SI)
    Energyvec.append(energy)
    ratio=(np.mean(Qvec))/energy
    Ratiovec.append(ratio)
    Qvec_acum.append(np.mean(Qvec))
    print("Summary-Sampling interval:",SI,"SI-Stress:",np.mean(Qvec))
print(Qvec_acum)
plotterquality(SIvec,Qvec_acum,Energyvec)
plotter2(timevec,wcvec,wcthreshold,readingvec)




2010 5 0.008853746990230319 2.586206896551724
2011 5 0.00943824884264558 3.0357142857142856
2012 5 0.005435040303508137 1.25
2013 5 0.00864792130502848 2.5
2014 5 0.007608735893496698 2.2413793103448274
2015 5 0.009728739187395228 2.8333333333333335
2016 5 0.007029406724552081 1.9642857142857142
2017 5 0.009200084674633265 2.34375
2018 5 0.008957835628878247 2.5
2019 5 0.009451130807987948 2.586206896551724
Summary-Sampling interval: 5 SI-Stress: 0.008435089035835598
2010 10 0.016049634959381817 3.9655172413793105
2011 10 0.020230074190523716 5.535714285714286
2012 10 0.018941392744412755 4.821428571428571
2013 10 0.014520795977517337 4.230769230769231
2014 10 0.01905422513334668 5.172413793103448
2015 10 0.023034416269852868 5.833333333333333
2016 10 0.017874457885571845 4.285714285714286
2017 10 0.021041671307299052 4.84375
2018 10 0.022489948673131005 5.892857142857143
2019 10 0.023965780721844722 5.862068965517241
Summary-Sampling interval: 10 SI-Stress: 0.01972023978628818
2010 20

Loading BokehJS ...

Loading BokehJS ...

In [38]:
import numpy as np
import pandas as pd

color_map=pd.read_csv('color_map_lettuce.csv')
color_map=color_map.set_index("Stress", drop = False)
a=color_map.loc[99,"Factor"]
print(a)

1.1


In [12]:
date=initial_date
Kcdates=[]
for i in range (daystot+1):
    Kcdates.append(date)
    date = date+timedelta(days=1)
plotter1(Kcdates,Kc)
#print(Kcdates)
#plotterquality(SIvec,Qvec_acum)

Loading BokehJS ...

In [1]:
def plotter1(x,y):
    import numpy as np
    from bokeh.io import output_file, output_notebook
    from bokeh.plotting import figure, show
    output_notebook()
    fig = figure(background_fill_color='white',
                 border_fill_color='white',
                 x_axis_label='Date',
                 x_axis_type='datetime',
                 x_axis_location='below',
                 #x_range=(0, 10),
                 y_axis_label='Kc',
                 y_axis_type='linear',
                 y_axis_location='left',
                 y_range=(0.2, 1),
                 title='Kc for vineyard',
                 title_location='right',
                 toolbar_location='below',
                 tools=['save', 'undo', 'box_zoom'])
    fig.line(x=x, y=y,color='green', line_width=3)
    show(fig)

def plotter2(x,y,wcthreshold,readingvec):
    import numpy as np
    from bokeh.io import output_file, output_notebook
    from bokeh.plotting import figure, show
    output_notebook()
    fig = figure(background_fill_color='white',
                 border_fill_color='white',
                 x_axis_label='Date',
                 x_axis_type='datetime',
                 x_axis_location='below',
                 #x_range=(0, 10),
                 y_axis_label='Volumetric water content',
                 y_axis_type='linear',
                 y_axis_location='left',
                 y_range=(0, 1),
                 #title='Kc',
                 title_location='right',
                 toolbar_location='below',
                 tools=['save', 'undo', 'box_zoom'])
    fig.line(x=x, y=y,color='green', line_width=1)
    fig.line(x=x, y=wcthreshold,color='red', line_width=1)
    fig.circle(x=x, y=readingvec,color='blue')
    show(fig)
    
def plotter3(x,y):
    import numpy as np
    from bokeh.io import output_file, output_notebook
    from bokeh.plotting import figure, show
    #output_notebook()
    fig = figure(background_fill_color='white',
                 border_fill_color='white',
                 x_axis_label='Sampling frequency(min)',
                 #x_axis_type='log',
                 x_axis_location='below',
                 #x_range=(0, 10),
                 y_axis_label='Policy quality',
                 #y_axis_type='linear',
                 y_axis_location='left',
                 y_range=(0.95, 1),
                 #title='Kc',
                 title_location='right',
                 toolbar_location='below',
                 tools=['save', 'undo', 'box_zoom'])
    #fig.line(x=x, y=y,color='green', line_width=1)
    fig.circle(x=x, y=y,color='blue')
    show(fig)
    
def plotterquality(x,y,energy):
    import numpy as np
    from bokeh.io import output_file, output_notebook
    from bokeh.plotting import figure, show
    output_notebook()
    fig = figure(background_fill_color='white',
                 border_fill_color='white',
                 x_axis_label='Sampling interval',
                 #x_axis_type='datetime',
                 x_axis_location='below',
                 #x_range=(0, 10),
                 y_axis_label='Policyquality',
                 y_axis_type='linear',
                 y_axis_location='left',
                 y_range=(0, 1),
                 #title='Kc',
                 title_location='right',
                 toolbar_location='below',
                 tools=['save', 'undo', 'box_zoom'])
    fig.line(x=x, y=y,color='green', line_width=2, legend_label="Policy quality")
    fig.line(x=x, y=energy,color='red', line_width=2, legend_label="Energy used")
    #fig.line(x=x, y=ratio,color='blue', line_width=2, legend_label="ratio")
    show(fig)
